In [3]:
# Урок 3. Парсинг данных. HTML, Beautiful Soap
# Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру. 
# Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:
# vv Наименование вакансии.
# vv Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
# vv Ссылку на саму вакансию.
# Сайт, откуда собрана вакансия.
# По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). 
# Структура должна быть одинаковая для вакансий с всех сайтов. 
# Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

# Урок 4. Добавляем найденные вакансии в базу mongodb

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient

In [4]:
# Функция парсит вакансии с hh.ru

def fParseHhRu(vacancies_list, vacancy_text, pagenum):
    url = 'https://hh.ru/search/vacancy'
    params = {  'area': 1,
                'text': vacancy_text,
                'items_on_page': 20,
                'experience': 'noExperience',
                'page': pagenum
             }
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    
    res = requests.get(url=url, params=params, headers=headers) 
    soup = bs(res.text, 'html.parser')
    vacancies = soup.find_all('div', {'class': 'serp-item'})
    #vacancies_list = []

    for vacancy in vacancies:
        vacancy_name = vacancy.find('a', {'class': 'serp-item__title'}).getText()
        vacancy_link = vacancy.find('a', {'class': 'serp-item__title'})['href']
        vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        vacancy_employer = vacancy.find('a', {'class': 'bloko-link bloko-link_kind-tertiary'}).getText()
        vacancy_adress = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText()
        if vacancy_salary is not None:
                vacancy_salary = vacancy_salary.getText().replace('\u202f','')
        vacancy_dict = {  'source': 'hh.ru',
                          'name': vacancy_name,
                          'link': vacancy_link,
                          'salary': vacancy_salary,
                          'employer': vacancy_employer,
                          'adress': vacancy_adress
                       }
        vacancies_list.append(vacancy_dict)
    #return vacancies_list

In [51]:
# Функция парсит вакансии с rabota.ru

def fParseRabotaRu(vacancies_list, vacancy_text, pagenum):
    url = 'https://www.rabota.ru/vacancy/'
    params = { 'query': vacancy_text,
               'sort': 'relevance',
               'page': pagenum
             }
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

    res = requests.get(url=url, params=params, headers=headers) 
    soup = bs(res.text, 'html.parser')
    vacancies = soup.find_all('div', {'class': 'vacancy-preview-card__top'})
    
    for vacancy in vacancies:
            vacancy_name = vacancy.find('a', {'class': 'vacancy-preview-card__title_border'}).getText()
            vacancy_link = vacancy.find('a', {'class': 'vacancy-preview-card__title_border'})['href']
            if "https://" not in vacancy_link:
                vacancy_link = "https://rabota.ru" + vacancy_link
            if "?search_id=" in vacancy_link:
                vacancy_link = vacancy_link[0:vacancy_link.rfind('/')+1]
            vacancy_salary = vacancy.find('div', {'class': 'vacancy-preview-card__salary'}).find('a')
            vacancy_employer = vacancy.find('a', {'itemprop': 'name'}).getText()
            adr=vacancy.find('span', {'class': 'vacancy-preview-location__address-text'})
            if(adr is not None):
                vacancy_adress = adr.getText()
   
            if vacancy_salary is not None:
                vacancy_salary = vacancy_salary.getText().replace('&nbsp;','')

            vacancy_dict = { 'source': 'rabota.ru',
                             'name': vacancy_name.replace('  ','').replace('\n',''),
                             'link': vacancy_link,
                             'salary': vacancy_salary,
                             'employer': vacancy_employer.replace('  ','').replace('\n',''),
                             'adress': vacancy_adress.replace('  ','').replace('\n','')
                           }
            vacancies_list.append(vacancy_dict)

In [52]:
# Функция добавляет документ в базу, предварительно проверяя условие check
# Если запись с таким условием есть - повторно не добавляет.
def mongo_insert_document(collection, data, check):
    res = collection.find(check)
    if(list(res)!=[]):
        return
    return collection.insert_one(data).inserted_id

In [54]:
# запросы к сайтам и формирование списка вакансий
# добавление документов в базу mongodb

mongo_client = MongoClient('mongodb://localhost:27017/')
db = mongo_client.jobs_db

vacancies_list = []
vacancy_text='python'

fParseHhRu(vacancies_list,vacancy_text,0)
fParseHhRu(vacancies_list,vacancy_text,1)
fParseHhRu(vacancies_list,vacancy_text,2)

fParseRabotaRu(vacancies_list,vacancy_text,1)
fParseRabotaRu(vacancies_list,vacancy_text,2)
fParseRabotaRu(vacancies_list,vacancy_text,3)

#db.vacancies.drop()

for v in vacancies_list:
    id = mongo_insert_document(db.vacancies,v,{'link':v.get('link')})
    if(id is None):
        print('Вакансия уже есть в нашей базе.')
    else:
        print(f'Вакансия добавлена, id: {id}, link: {v.get("link")}')
        
print(f'Всего вакансий в базе: { db.vacancies.count_documents({}) }')


Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия уже есть в нашей базе.
Вакансия

In [ ]:
# вывод списка в csv файл
#vacancies_df = pd.DataFrame(vacancies_list)
#vacancies_df.to_csv('vacancies_list.csv')

In [27]:
for v in list(db.vacancies.find()):
    print(v)

{'_id': ObjectId('634fee20e5f64094d1a4436b'), 'source': 'hh.ru', 'name': 'ML Researcher', 'link': 'https://hh.ru/vacancy/71202794?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'salary': '250000 – 350000 руб.', 'employer': 'Gara.capital', 'adress': 'Москва'}
{'_id': ObjectId('634fee20e5f64094d1a4436c'), 'source': 'hh.ru', 'name': 'Разработчик (JS, Python)', 'link': 'https://hh.ru/vacancy/70343886?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'salary': None, 'employer': 'Национальный исследовательский университет Высшая школа экономики', 'adress': 'Москва'}
{'_id': ObjectId('634fee20e5f64094d1a4436d'), 'source': 'hh.ru', 'name': 'Консультант-разработчик (Cognos/Optimacros)', 'link': 'https://hh.ru/vacancy/71027991?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'salary': None, 'employer': 'Human Line', 'adress': 'Москва, Библиотека им.Ленина'}
{'_id': ObjectId('634fee20e5f64094d1a4436e'), 'source': 'hh.ru', 'name': 'Разра